# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [3]:
# TODO: Create a .env file with the following variables
# OPENAI_API_KEY="YOUR_KEY"
# CHROMA_OPENAI_API_KEY="YOUR_KEY"
# TAVILY_API_KEY="YOUR_KEY"

In [4]:
# TODO: Load environment variables
load_dotenv()

True

### VectorDB Instance

In [5]:
# TODO: Instantiate your ChromaDB Client
# Choose any path you want
chroma_client = chromadb.PersistentClient(path="lib/persistent_memory")

### Collection

In [6]:
# TODO: Pick one embedding function
# If picking something different than openai, 
# make sure you use the same when loading it
embedding_fn = embedding_functions.OpenAIEmbeddingFunction()

In [7]:
# TODO: Create a collection
# Choose any name you want
# collection = chroma_client.create_collection(
#    name="udaplay",
#    embedding_function=embedding_fn
#)
game_collect = chroma_client.create_collection(name="udaplay", embedding_function=embedding_fn)

### Add documents

In [8]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    game_collect.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

In [9]:
game_collect.count()

15

In [15]:
game_collect.query(
    query_texts=["Princess Sensor"],
    n_results=2,
    include=['metadatas', 'documents', 'distances']
)

{'ids': [['013', '009']],
 'embeddings': None,
 'documents': [['[Xbox 360] Kinect Adventures! (2010) - A collection of mini-games designed to showcase the capabilities of the Kinect motion sensor.',
   "[Nintendo 64] Super Mario 64 (1996) - A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach."]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'Name': 'Kinect Adventures!',
    'Genre': 'Party',
    'YearOfRelease': 2010,
    'Description': 'A collection of mini-games designed to showcase the capabilities of the Kinect motion sensor.',
    'Platform': 'Xbox 360',
    'Publisher': 'Microsoft Game Studios'},
   {'Name': 'Super Mario 64',
    'Platform': 'Nintendo 64',
    'Genre': 'Platformer',
    'Publisher': 'Nintendo',
    'Description': "A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.",
    'YearO

In [16]:
result = game_collect.query(
    query_texts=["gadget"],
    n_results=2,
    include=['metadatas', 'documents', 'distances']
)

result['documents'][0]

['[Xbox 360] Kinect Adventures! (2010) - A collection of mini-games designed to showcase the capabilities of the Kinect motion sensor.',
 '[PlayStation 3] Gran Turismo 5 (2010) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.']

In [18]:
size = len(game_collect.peek(1)['embeddings'][0])
print(f"Size of the embeddings array: {size}")

Size of the embeddings array: 1536
